In [12]:
import matplotlib.pyplot as plt

def draw_number(pixels):
    plt.figure()
    plt.imshow(pixels)
    plt.colorbar()
    plt.grid(False)
    plt.show()
    
def load_emnist(key_path):
    import os
    if not os.path.isfile("~/.kaggle/kaggle.json"):
        !mv {key_path} ~/.kaggle/kaggle.json
        !chmod 600 ~/.kaggle/kaggle.json
        !pip install kaggle
        !kaggle datasets download -d crawford/emnist -p data
        
        import zipfile
        with zipfile.ZipFile("data/emnist.zip", 'r') as zip_ref:
            zip_ref.extractall("data/emnist")

In [13]:
load_emnist("kaggle.json")

mv: cannot stat 'kaggle.json': No such file or directory
emnist.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import tensorflow as tf

mnist_raw = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist_raw.load_data()

label_names = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

x_train = x_train / 255.0
x_test = x_test / 255.0

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax(),
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=9)

_, test_acc = model.evaluate(x_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

Epoch 1/9
1875/1875 [==============================] - 1s 450us/step - loss: 1.5705 - accuracy: 0.9065
Epoch 2/9
1875/1875 [==============================] - 1s 452us/step - loss: 1.5150 - accuracy: 0.9506
Epoch 3/9
1875/1875 [==============================] - 1s 446us/step - loss: 1.5016 - accuracy: 0.9633
Epoch 4/9
1875/1875 [==============================] - 1s 449us/step - loss: 1.4946 - accuracy: 0.9692
Epoch 5/9
1875/1875 [==============================] - 1s 447us/step - loss: 1.4889 - accuracy: 0.9745
Epoch 6/9
1875/1875 [==============================] - 1s 449us/step - loss: 1.4851 - accuracy: 0.9783
Epoch 7/9
1875/1875 [==============================] - 1s 451us/step - loss: 1.4827 - accuracy: 0.9805
Epoch 8/9
1875/1875 [==============================] - 1s 445us/step - loss: 1.4802 - accuracy: 0.9827
Epoch 9/9
1875/1875 [==============================] - 1s 447us/step - loss: 1.4783 - accuracy: 0.9845
313/313 - 0s - loss: 1.4896 - accuracy: 0.9734 - 101ms/epoch - 323us/step